<a href="https://colab.research.google.com/github/mihhwiNick/DA_project/blob/main/EDA-Benh_Dai-Thao-Duong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sử dụng thuật toán ADAP để dự đoán bệnh tiểu đường**

##**1.Định nghĩa vấn đề**
+ **Mô tả**:
   + Bộ dữ liệu gồm 768 mẫu từ bệnh nhân đều là nữ giới, ít nhất 21 tuổi, và có nguồn gốc người da đỏ Pima, sống gần Phoenix, Arizona.
   + 8 đặc điểm được đo từ mỗi mẫu + 1 biến phân loại (có/không mắc tiểu đường)
+ **Dữ liệu vào**:
   + Số lần mang thai (lần)
   + Nồng độ glucose huyết tương sau 2 giờ trong xét nghiệm dung nạp glucose đường uống (mg/dL)
   + Huyết áp tâm trương (mm Hg)
   + Độ dày nếp gấp da cơ tam đầu (mm)
   + Nồng độ insulin huyết thanh sau 2 giờ (mu U/ml)
   + Chỉ số khối cơ thể BMI (kg/m²)
   + Hàm phả hệ tiểu đường ()
   + Tuổi (năm)
   + Biến phân loại (nhị phân)
+ **Dữ liệu ra**:
   + Dự đoán bệnh nhân có tiểu đường không (Biến phân loại 0 hoặc 1)
+ **Mục tiêu**:
   + Phân tích bộ dữ liệu bệnh đái tháo đường nhằm hiểu rõ các yếu tố liên quan đến nguy cơ mắc bệnh.
   + Vận dụng kiến thức về dinh dưỡng để đánh giá ảnh hưởng của chế độ ăn uống đến tình trạng bệnh.


## **2.Chuẩn bị vấn đề**

### **2.1.Khai báo thư viện**

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.model_selection as mod
import sklearn.neighbors as nei
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report, precision_recall_curve, average_precision_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import operator
%matplotlib inline

### **2.2.Nạp dữ liệu**

In [ ]:
#load data
pima = pd.read_csv('https://raw.githubusercontent.com/mihhwiNick/DA_project/refs/heads/main/pima-indians-diabetes.data.csv')

## **3.Phân tích khám phá dữ liệu**

### **3.1.Phân tích đơn biến (Univariable Analysis)**

#### **3.1.1. Hiển thị một số thông tin về dữ liệu**
+ Số dòng, số cột của dữ liệu
+ Kiểu dữ liệu của từng cột
+ 5 dòng đầu và 5 dòng cuối của bảng dữ liệu
+ Thông tin chung về dữ liệu

In [ ]:
# Get shape
print(f'+ Shape: {pima.shape}')

# Get types
print(f'\n+ Data Types:\n{pima.dtypes}')

# Check the first 5 rows (head)
print('\n+ First 5 rows:')
display(pima.head(5))

# Check the last 5 rows (tail)
print('\n+ Last 5 rows:')
display(pima.tail(5))

# info
print('\n + Basic Information: ')
pima.info()

+ Shape: (768, 9)

+ Data Types:
Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

+ First 5 rows:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1



+ Last 5 rows:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0



 + Basic Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


**Nhận xét**:
+ Dữ liệu có 8 tính chất để phân lớp: Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age
+ Các giá trị đều là số học, thuộc kiểu int64 và float64
+ Tổng số dòng dữ liệu là 768 dòng
+ Dữ liệu phân lớp nằm ở cột Outcome, gồm 2 giá trị: 0 (không mắc bệnh) và 1 (mắc bệnh)


#### **3.1.2. Kiểm tra tính toàn vẹn của dữ liệu**
+ Dữ liệu có bị trùng lặp không? Hiển thị dòng bị vi phạm.
+ Dữ liệu có tồn tại giá trị Null không? Hiển thị dòng bị vi phạm.
+ Dữ liệu có tồn tại giá trị NaN không? Hiển thị dòng bị vi phạm.

In [ ]:
# Loại bỏ cột Outcome khi kiểm tra giá trị 0
cols_to_check = pima.columns.drop("Outcome")
rows_with_zero = (pima[cols_to_check] == 0).any(axis=1)
count_rows_with_zero = rows_with_zero.sum()

has_null = pima.isnull().sum().any()
has_nan  = pima.isna().sum().any()
n_duplicated = pima.duplicated().sum()

print('Tính toàn vẹn dữ liệu:')
print(f'+ Có giá trị Null: {has_null}')
if has_null:
    display(pima[pima.isnull().any(axis=1)])

print(f'+ Có giá trị NaN: {has_nan}')
if has_nan:
    display(pima[pima.isna().any(axis=1)])

print(f'+ Số dòng trùng: {n_duplicated}')
if n_duplicated > 0:
    s_query1 = "`Pregnancies`==6 and `Glucose`==148 and `Outcome`==1"
    s_query2 = "`Pregnancies`==1 and `Glucose`==85 and `Outcome`==0"
    display(pima.query(s_query1))
    display(pima.query(s_query2))
    display(pima[pima.duplicated()])

zero_counts_all = (pima == 0).sum()


cols_check = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

# Đếm số dòng có giá trị = 0 ở các cột này
zero_counts = (pima[cols_check] == 0).sum()

print("Số lượng dòng có giá trị = 0 ở từng cột được chọn:")
print(zero_counts)

Tính toàn vẹn dữ liệu:
+ Có giá trị Null: False
+ Có giá trị NaN: False
+ Số dòng trùng: 0
Số lượng dòng có giá trị = 0 ở từng cột được chọn:
Glucose            5
BloodPressure     35
SkinThickness    227
Insulin          374
BMI               11
dtype: int64


**Nhận xét**:
+ Dữ liệu không có giá trị rỗng (NaN, Null).
+ Tuy nhiên, khi kiểm tra chi tiết, phát hiện một số cột có giá trị = 0 không hợp lý về mặt y học, được coi là dữ liệu thiếu:
  + Glucose: không thể bằng 0 vì cơ thể luôn có đường huyết lưu hành.

  + BloodPressure: không thể bằng 0, vì huyết áp 0 chỉ xảy ra khi đã tử vong.

  + SkinThickness: không thể bằng 0, lớp mỡ dưới da luôn có độ dày nhất định.

  + Insulin: không thể bằng 0, vì tuyến tụy luôn tiết insulin nền.

  + BMI: không thể bằng 0, vì chỉ số khối cơ thể = 0 nghĩa là cân nặng = 0, điều này không thực tế.

#### **3.1.3. Các tính chất thống kê trên dữ liệu số**
+ Count, Mean, Standard Deviation, Minimum Value
+ 25th Percentile, 50th Percentile (Median), 75th Percentile, Maximum Value

In [ ]:
description = pima.drop(columns=["Outcome"]).describe().T
display(description)

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00


**Nhận xét**:
+ **Pregnancies:** Trung bình 3.84, dao động 0–17 → hợp lý, vì có thể có phụ nữ chưa mang thai.
+ **Glucose, BloodPressure, SkinThickness, Insulin, BMI:** Xuất hiện nhiều giá trị = 0, không hợp lý về mặt y học → đây là dữ liệu thiếu được mã hoá bằng 0, cần xử lý ở bước làm sạch dữ liệu. Trong đó, Insulin còn có outlier rất lớn (846).
+ **DiabetesPedigreeFunction:** Dao động 0.078–2.42, không thấy bất thường rõ ràng.

+ **Age:** Trung bình 33 tuổi, dao động 21–81 → hợp lý với dân số khảo sát.

#### **3.1.4. Tần số xuất hiện trên dữ liệu phân loại**

In [ ]:
pima["Outcome"].value_counts()

,count
Outcome,
0,500
1,268


**Nhận xét**:
+ Dữ liệu không cân bằng: lớp 0 chiếm khoảng 65%, lớp 1 chiếm khoảng 35%, điều này có thể làm mô hình thiên lệch về lớp đa số.
+ **Giải pháp**: có thể áp dụng oversampling (SMOTE) để tăng số mẫu lớp 1, hoặc undersampling để giảm số mẫu lớp 0

### **3.2.Phân tích đa biến (Multivariable Analysis)**

#### **3.2.1. So sánh trung bình các đặc điểm theo nhóm Outcome**

In [ ]:
# Tính giá trị trung bình của các biến theo nhóm Outcome
pima.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


**Nhận xét**: Kết quả cho thấy nhóm phụ nữ mắc tiểu đường có xu hướng sở hữu các đặc điểm sinh lý với giá trị trung bình cao hơn so với nhóm không mắc bệnh.

In [ ]:
# Lấy giá trị trung bình của từng biến trong dataset
pima.mean()

,0
Pregnancies,3.845052
Glucose,120.894531
BloodPressure,69.105469
SkinThickness,20.536458
Insulin,79.799479
BMI,31.992578
DiabetesPedigreeFunction,0.471876
Age,33.240885
Outcome,0.348958


**Nhận xét**: Vì một số phụ nữ không có thông tin về một số đặc điểm như glucose, huyết áp, độ dày da, Insulin và chỉ số BMI, nên giá trị trung bình của các đặc điểm này có thể không chính xác. Để tính toán giá trị trung bình đúng, chỉ nên bao gồm những người có giá trị lớn hơn 0.

##### **3.2.1.1. So sánh trung bình Glucose theo nhóm Outcome**

In [ ]:
# Lọc những phụ nữ có giá trị Glucose khác 0
pima_glucose = pima.loc[pima['Glucose'] != 0]

# Hiển thị kích thước của tập dữ liệu sau khi lọc
pima_glucose.shape

(763, 9)

In [ ]:
# Đếm số dòng dữ liệu có giá trị Glucose bằng 0
(pima['Glucose'] == 0).sum()

np.int64(5)

In [ ]:
# Tính giá trị trung bình Glucose của tập dữ liệu sau khi lọc
pima_glucose['Glucose'].mean()

np.float64(121.6867627785059)

In [ ]:
# Tính giá trị trung bình các đặc điểm theo nhóm Outcome
pima_glucose.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.311871,110.643863,68.213280,19.631791,69.160966,30.317304,0.430662,31.247485
1,4.860902,142.319549,70.800752,22.056391,101.090226,35.106015,0.550605,37.052632


In [ ]:
# Tính giá trị lớn nhất của Glucose
pima_glucose['Glucose'].max()  # 199

199

In [ ]:
# Tính giá trị nhỏ nhất của Glucose
pima_glucose['Glucose'].min()

44

In [ ]:
pima_glucose[pima_glucose['Glucose'] == 44]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
62,5,44,62,0,0,25.0,0.587,36,0


**Nhận xét**:
+ Trong tổng số 768 người, có 763 người có dữ liệu Glucose hợp lệ, chỉ 5 người bị thiếu thông tin.
+ Giá trị trung bình Glucose của nhóm hợp lệ là khoảng 121.69 mg/dL, nằm trong ngưỡng bình thường (<140 mg/dL).
+ Khi phân nhóm theo Outcome, người mắc tiểu đường có mức Glucose trung bình cao hơn rõ rệt (142.32 so với 110.64).
+ Giá trị Glucose cao nhất là 199 mg/dL, được xem là mức nguy hiểm đối với bệnh tiểu đường.


##### **3.2.1.2. So sánh trung bình BloodPressure theo nhóm Outcome**

In [ ]:
# Lọc những phụ nữ có giá trị BloodPressure khác 0
pima_BloodPressure = pima.loc[pima['BloodPressure'] != 0]

# Hiển thị kích thước của tập dữ liệu sau khi lọc
pima_BloodPressure.shape

(733, 9)

In [ ]:
# Đếm số dòng dữ liệu có giá trị BloodPressure bằng 0
(pima['BloodPressure'] == 0).sum()

np.int64(35)

In [ ]:
# Tính giá trị trung bình BloodPressure của tập dữ liệu sau khi lọc
pima_BloodPressure['BloodPressure'].mean()

np.float64(72.40518417462484)

In [ ]:
# Tính giá trị nhỏ nhất của BloodPressure
pima_BloodPressure['BloodPressure'].min()

24

In [ ]:
# Tính giá trị lớn nhất của BloodPressure
pima_BloodPressure['BloodPressure'].max()

122

**Nhận xét**:
+ Trong tổng số 768 người, chỉ có 733 người có dữ liệu huyết áp tâm trương(BloodPressure) hợp lệ, tức là 35 người bị thiếu thông tin.
+ Giá trị huyết áp tâm trương(BloodPressure) cao nhất là >90 mmHg, cho thấy một số người có khả năng bị tăng huyết áp theo chuẩn chẩn đoán ≥140/90 mmHg (NICE, 2019).
+ Người mắc tiểu đường nên được điều trị để duy trì huyết áp tâm trương(BloodPressure) dưới 90 mmHg, nhằm giảm nguy cơ biến chứng.
+ Theo khuyến nghị của NICE (2019), người tiểu đường type 2 không có tiền sử tăng huyết áp hoặc bệnh thận nên được đo huyết áp ít nhất mỗi năm.
+ Bộ dữ liệu không có thông tin về huyết áp tâm thu(Systolic), trong khi các nghiên cứu cho thấy huyết áp ≥115/75 mmHg có liên quan đến nhiều bệnh lý nghiêm trọng (BOER et al., 2017).
+ Giá trị huyết áp tâm trương thấp nhất là 24 mmHg, hoàn toàn nằm ngoài ngưỡng bình thường (60–80 mmHg). Đây có thể là lỗi nhập liệu, hoặc phản ánh tình trạng huyết áp quá thấp cần được theo dõi.
+ Người có huyết áp tâm trương ≤60 mmHg được xem là huyết áp thấp, cần được theo dõi kỹ để tránh nguy cơ suy tuần hoàn.

##### **3.2.1.3. So sánh trung bình Insulin theo nhóm Outcome**

In [ ]:
# Lọc những phụ nữ có giá trị Insulin khác 0
pima_insulin = pima.loc[pima['Insulin'] != 0]

# Hiển thị kích thước của tập dữ liệu sau khi lọc
pima_insulin.shape

(394, 9)

In [ ]:
# Đếm số dòng dữ liệu có giá trị Insulin bằng 0
(pima['Insulin'] == 0).sum()

np.int64(374)

In [ ]:
# Tính giá trị trung bình Insulin của tập dữ liệu sau khi lọc
pima_insulin['Insulin'].mean()

np.float64(155.5482233502538)

In [ ]:
# Tính giá trị nhỏ nhất của Insulin
pima_insulin['Insulin'].min()

14

In [ ]:
# Tính giá trị lớn nhất của Insulin
pima_insulin['Insulin'].max()

846

**Nhận xét**:
+ Trong tổng số 768 người, có 394 người có dữ liệu Insulin hợp lệ, tức là 374 người bị thiếu thông tin (Insulin = 0).
+ Giá trị thấp nhất là 14 mIU/L, hơi thấp hơn ngưỡng bình thường, có thể phản ánh tình trạng thiếu insulin hoặc sai số đo.
+ Ngưỡng bình thường của insulin sau 2 giờ là từ 16 đến 166 mIU/L, theo các tài liệu y khoa.
+ Giá trị trung bình insulin của mẫu dữ liệu nằm trong ngưỡng bình thường, cho thấy phần lớn người tham gia không có dấu hiệu rối loạn insulin rõ rệt.
+ Tuy nhiên, một số phụ nữ trong mẫu có mức insulin cao hơn bình thường, có thể là dấu hiệu của:
  + Béo phì (obesity)
  + Giai đoạn sớm của tiểu đường type 2 (T2DM)
  + Sử dụng insulin quá mức hoặc điều trị bằng insulin
+ Những trường hợp có insulin cao cần được phân tích kỹ hơn, kết hợp với các đặc điểm khác như BMI, Glucose và Outcome để đánh giá nguy cơ bệnh lý.


##### **3.2.1.4. So sánh trung bình BMI theo nhóm Outcome**

In [ ]:
# Lọc những phụ nữ có giá trị Insulin khác 0
pima_BMI = pima.loc[pima['BMI'] != 0]

# Hiển thị kích thước của tập dữ liệu sau khi lọc
pima_BMI.shape

(757, 9)

In [ ]:
# Đếm số dòng dữ liệu có giá trị BMI bằng 0
(pima['BMI'] == 0).sum()

np.int64(11)

In [ ]:
# Tính giá trị trung bình BMI của tập dữ liệu sau khi lọc
pima_BMI['BMI'].mean()

np.float64(32.457463672391015)

In [ ]:
# Tính giá trị nhỏ nhất của BMI
pima_BMI['BMI'].min()

18.2

In [ ]:
# Tính giá trị lớn nhất của BMI
pima_BMI['BMI'].max()

67.1

**Nhận xét**:
+ Trong tổng số 768 người, có 757 người có dữ liệu BMI hợp lệ, tức là chỉ 11 người bị thiếu thông tin (BMI = 0).
+ Giá trị trung bình BMI của nhóm hợp lệ là khoảng 32.46 kg/m², nằm trong ngưỡng béo phì theo phân loại của Tổ chức Y tế Thế giới (WHO: BMI ≥ 30).
+ Giá trị BMI thấp nhất là 18.2 kg/m², cho thấy có ít nhất 1 trường hợp bị thiếu cân (underweight: BMI < 18.5).
+ Giá trị BMI cao nhất là 67.1 kg/m², vượt xa ngưỡng béo phì bệnh lý (morbid obesity: BMI ≥ 40), cho thấy có một số trường hợp nguy cơ sức khỏe nghiêm trọng.
+ Phân bố BMI trong tập dữ liệu cho thấy xu hướng thừa cân và béo phì chiếm tỷ lệ cao, cần được phân tích sâu hơn theo nhóm Outcome để đánh giá mối liên hệ với nguy cơ tiểu đường.




##### **3.2.1.5. So sánh tổng hợp các đặc điểm sức khỏe của phụ nữ có đầy đủ dữ liệu**


In [ ]:
# Lọc những phụ nữ có các giá trị Glucose, BloodPressure, Insulin và BMI khác 0
pima_all = pima.loc[(pima['BMI'] != 0) & (pima['Insulin'] != 0) & (pima['BloodPressure'] != 0) & (pima['Glucose'] != 0)]

# Hiển thị kích thước của tập dữ liệu sau khi lọc
pima_all.shape

(392, 9)

**Nhận xét**: Chỉ có 392 phụ nữ trong tập dữ liệu có đầy đủ thông tin ở tất cả các đặc điểm. Con số này chiếm chưa đến một nửa tổng mẫu, tức khoảng 49% số phụ nữ trong tập dữ liệu có đầy đủ thông tin ở tất cả các thuộc tính.


In [ ]:
# Đếm số dòng dữ liệu có các giá trị Glucose, BloodPressure, Insulin và BMI bằng 0
((pima['Glucose'] == 0) | (pima['BloodPressure'] == 0) | (pima['Insulin'] == 0) | (pima['BMI'] == 0)).sum()

np.int64(376)

In [ ]:
pima_all['Outcome'].value_counts()

,count
Outcome,
0,262
1,130


**Nhận xét**: Có 262 phụ nữ không mắc bệnh tiểu đường (DM) và 130 phụ nữ mắc bệnh tiểu đường trong nhóm có đầy đủ thông tin về các đặc điểm sức khỏe. Điều này cho thấy tỷ lệ mắc bệnh trong nhóm này là khoảng 33%.


In [ ]:
pima_all.mean()

,0
Pregnancies,3.301020
Glucose,122.627551
BloodPressure,70.663265
SkinThickness,29.145408
Insulin,156.056122
BMI,33.086224
DiabetesPedigreeFunction,0.523046
Age,30.864796
Outcome,0.331633


In [ ]:
pima_all.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,2.721374,111.431298,68.969466,27.251908,130.854962,31.750763,0.472168,28.347328
1,4.469231,145.192308,74.076923,32.961538,206.846154,35.777692,0.625585,35.938462


**Nhận xét**:
+ Phụ nữ mắc bệnh tiểu đường có xu hướng số lần mang thai cao hơn, cùng với các chỉ số sức khỏe như Glucose, huyết áp tâm trương(BloodPressure), độ dày da(SkinThickness), Insulin, BMI, chỉ số di truyền tiểu đường (Diabetes Pedigree Function) và tuổi đều cao hơn so với nhóm không mắc bệnh.
+ Cả hai nhóm đều có giá trị BMI trung bình ≥ 30, cho thấy tình trạng béo phì phổ biến trong toàn bộ mẫu dữ liệu.
+ Nhóm phụ nữ mắc tiểu đường có giá trị Insulin trung bình vượt ngưỡng bình thường (trên 166 mIU/L), phản ánh tình trạng rối loạn chuyển hóa hoặc điều trị bằng insulin.
+ Cả hai nhóm đều có giá trị Glucose trung bình > 100 mg/dL, vượt ngưỡng bình thường, cho thấy nguy cơ tiền tiểu đường ở cả nhóm chưa mắc bệnh.
+ Điều này có thể cho thấy rằng một số phụ nữ chưa mắc bệnh tiểu đường vẫn có nguy cơ cao trong tương lai, đặc biệt là những người có mức Insulin cao, có thể liên quan đến tình trạng kháng insulin.


In [ ]:
pima_all.min()

,0
Pregnancies,0.000
Glucose,56.000
BloodPressure,24.000
SkinThickness,7.000
Insulin,14.000
BMI,18.200
DiabetesPedigreeFunction,0.085
Age,21.000
Outcome,0.000


In [ ]:
pima_all.max()

,0
Pregnancies,17.00
Glucose,198.00
BloodPressure,110.00
SkinThickness,63.00
Insulin,846.00
BMI,67.10
DiabetesPedigreeFunction,2.42
Age,81.00
Outcome,1.00


In [ ]:
diabetic_malnourished = ((pima_all ['SkinThickness'] < 15.8) & (pima_all ['Outcome'] == 1)).sum()
diabetic_malnourished

np.int64(4)

**Nhận xét**: Có 4 phụ nữ trong số 130 người mắc bệnh tiểu đường bị suy dinh dưỡng, được xác định dựa trên tiêu chí độ dày da (SkinThickness) < 15.8 mm. Đây là tỷ lệ rất nhỏ, nhưng vẫn cho thấy có sự tồn tại của tình trạng dinh dưỡng kém trong nhóm bệnh nhân tiểu đường.


In [ ]:
diabetic_overweight_obese = ((pima_all ['BMI'] <= 30) & (pima_all ['Outcome'] == 1)).sum()
diabetic_overweight_obese

np.int64(20)

**Nhận xét**: Có 20 phụ nữ trong số 130 người mắc bệnh tiểu đường thuộc nhóm thừa cân hoặc béo phì, được xác định dựa trên chỉ số BMI ≤ 30 kg/m². Mặc dù ngưỡng béo phì thường bắt đầu từ BMI ≥ 30, nhưng nhóm này bao gồm cả những người đang ở ngưỡng thừa cân và béo phì nhẹ, cho thấy tình trạng dinh dưỡng không tối ưu vẫn phổ biến trong nhóm bệnh nhân tiểu đường.



#### **3.2.2. Phân loại tình trạng dinh dưỡng theo BMI**

In [ ]:
# Tạo cột Nutritional Status
Nutritional_status = pd.Series([])

In [ ]:
# Nutritional Status dựa trên BMI
for i in range(len(pima)):
    if pima['BMI'][i] == 0.0:
        Nutritional_status[i]="Không có dữ liệu"

    elif pima['BMI'][i] < 18.5:
        Nutritional_status[i]="Thiếu cân"

    elif pima['BMI'][i] < 25:
        Nutritional_status[i]="Bình thường"

    elif pima['BMI'][i] >= 25 and pima['BMI'][i] < 30:
        Nutritional_status[i]="Thừa cân"

    elif pima['BMI'][i] >= 30:
        Nutritional_status[i]="Béo phì"

    else:
        Nutritional_status[i]= pima['BMI'][i]

In [ ]:
# Thêm cột Nutritional Status
pima.insert(6, "Nutritional Status", Nutritional_status)

In [ ]:
# Kiểm tra dữ liệu sau khi thêm cột mới
pima.head (5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Nutritional Status,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,Béo phì,0.627,50,1
1,1,85,66,29,0,26.6,Thừa cân,0.351,31,0
2,8,183,64,0,0,23.3,Bình thường,0.672,32,1
3,1,89,66,23,94,28.1,Thừa cân,0.167,21,0
4,0,137,40,35,168,43.1,Béo phì,2.288,33,1


In [ ]:
# số lượng trạng thái dinh dưỡng
amount = pima['Nutritional Status'].value_counts()

# Tính phần trăm (dựa trên tổng số mẫu)
percentage = (amount / len(pima) * 100).round(2)

# Tạo DataFrame kết hợp số lượng và phần trăm
bang_ket_qua = pd.DataFrame({
    'Số lượng': amount,
    'Phần trăm (%)': percentage
})
print(bang_ket_qua)

                    Số lượng  Phần trăm (%)
Nutritional Status                         
Béo phì                  472          61.46
Thừa cân                 179          23.31
Bình thường              102          13.28
Không có dữ liệu          11           1.43
Thiếu cân                  4           0.52


**Nhận xét**:
+ Nhóm "Béo phì" chiếm tỷ lệ áp đảo (61.46%) trong dân số Pima, trong khi nhóm "Thiếu cân" chỉ chiếm 0.52%, phản ánh xu hướng thừa cân rõ rệt trong cộng đồng này.
+ Trong tổng số 768 phụ nữ, có 11 người không có thông tin về BMI. Chỉ 102 người có cân nặng bình thường và 4 người thuộc nhóm thiếu cân; phần lớn còn lại thuộc nhóm thừa cân hoặc béo phì, phản ánh xu hướng dinh dưỡng mất cân đối và cho thấy mối liên hệ tiềm năng giữa tình trạng dinh dưỡng và nguy cơ mắc tiểu đường.

#### **3.2.3. Phân loại tình trạng dinh dưỡng theo nồng độ Glucose huyết tương sau 2 giờ**

In [ ]:
# Tạo cột mới OGTT_Result (Kết quả xét nghiệm đường huyết sau khi uống glucose)
OGTT_Result = pd.Series([])

In [ ]:
# Phân loại kết quả xét nghiệm dung nạp glucose (OGTT) theo khuyến nghị của DIABETES UK (2019):
# - Nếu đường huyết sau 2 giờ < 140 mg/dL → Bình thường
# - Nếu từ 140 đến 199 mg/dL → Tiền tiểu đường (rối loạn dung nạp glucose)
# - Nếu ≥ 200 mg/dL → Tiểu đường type 2

for i in range(len(pima)):
    if pima['Glucose'][i] == 0.0:
        OGTT_Result [i]="NA"

    elif pima['Glucose'][i] <= 140:
        OGTT_Result [i]="Bình thường"

    elif pima['Glucose'][i] > 140 & pima['Glucose'][i] <= 198:
        OGTT_Result [i]="Rối loạn dung nạp glucose"

    elif pima['Glucose'][i] > 198:
        OGTT_Result[i]="Cấp độ tiểu đường"

    else:
        OGTT_Result [i]= pima['Glucose'][i]

In [ ]:
# Thêm cột Glucose Result
pima.insert(2, "Glucose Result", OGTT_Result)

In [ ]:
# Đếm số lượng từng nhóm kết quả glucose
vc_glucose = pima['Glucose Result'].value_counts()

# Tính phần trăm (dựa trên tổng số mẫu)
phần_trăm_glucose = (vc_glucose / len(pima) * 100).round(2)

# Tạo DataFrame kết hợp số lượng và phần trăm
bang_ket_qua_glucose = pd.DataFrame({
    'Số lượng': vc_glucose,
    'Phần trăm (%)': phần_trăm_glucose
})
print(bang_ket_qua_glucose)

                           Số lượng  Phần trăm (%)
Glucose Result                                    
Bình thường                     571          74.35
Rối loạn dung nạp glucose       192          25.00
NA                                5           0.65


In [ ]:
# Phân tích mối liên hệ giữa mức glucose và nguy cơ mắc tiểu đường

# Số người có glucose ở mức rối loạn dung nạp và mắc tiểu đường
so_nguoi_roi_loan_glucose_mac_benh = ((pima['Glucose'] > 140) & (pima['Glucose'] <= 198) & (pima['Outcome'] == 1)).sum()
print("Số người rối loạn dung nạp glucose và mắc tiểu đường:", so_nguoi_roi_loan_glucose_mac_benh)

# Số người có glucose bình thường nhưng vẫn mắc tiểu đường
so_nguoi_glucose_binh_thuong_mac_benh = ((pima['Glucose'] != 0) & (pima['Glucose'] <= 140) & (pima['Outcome'] == 1)).sum()
print("Số người glucose bình thường nhưng mắc tiểu đường:", so_nguoi_glucose_binh_thuong_mac_benh)

Số người rối loạn dung nạp glucose và mắc tiểu đường: 131
Số người glucose bình thường nhưng mắc tiểu đường: 134


**Nhận xét**:
+ Không có cá nhân nào trong tập dữ liệu có mức glucose sau uống đạt ngưỡng tiểu đường (≥ 198 mg/dL).
+ Có 131 người thuộc nhóm rối loạn dung nạp glucose (141–198 mg/dL) và đã được chẩn đoán mắc tiểu đường.
+ Có 134 người có mức glucose bình thường (≤ 140 mg/dL) nhưng vẫn mắc tiểu đường.
+ Điều này cho thấy: mức glucose cao không phải là yếu tố duy nhất gây bệnh, và tiểu đường có thể xuất hiện ngay cả khi glucose bình thường.
+ Khả năng tồn tại kháng insulin ở nhóm glucose bình thường là điều đáng chú ý.
+ *Việc* tính toán chỉ số HOMA-IR sẽ giúp đánh giá chính xác hơn mức độ kháng insulin trong mẫu nghiên cứu.





#### **3.2.4. Phân tích độ dày da theo độ tuổi**

In [ ]:
# Tạo biến để phân loại mức độ dày da
skin_level = pd.Series([])

In [ ]:
# Thống kê số lượng theo độ tuổi
pima['Age'].value_counts()

,count
Age,
22,72
21,63
25,48
24,46
23,38
28,35
26,33
27,32
29,29


In [ ]:
# Phân loại độ dày da theo phân vị chuẩn (P5, P10, ..., >P95)
for i in range(len(pima)):
    if pima["Age"][i] >= 20.0 and pima["Age"][i] <= 79.0:

        if pima["SkinThickness"][i] == 0.0:
            skin_level[i]=" 0 NA"

        elif pima["SkinThickness"][i] < 11.9:
            skin_level[i]="(1) <P5th"

        elif pima["SkinThickness"][i] == 11.9:
            skin_level[i]="(2) P5th"

        elif pima["SkinThickness"][i] > 11.9 and pima["SkinThickness"][i] < 14.0:
            skin_level[i]="(3) P5th - P10th"

        elif pima["SkinThickness"][i] == 14.0:
            skin_level[i]="(4) P10th"

        elif pima["SkinThickness"][i] > 14.0 and  pima["SkinThickness"][i] < 15.8:
            skin_level[i]="(5) P10th - P15th"

        elif pima["SkinThickness"][i] == 15.8:
            skin_level[i]="(6) P15th"

        elif pima["SkinThickness"][i] > 15.8 and pima["SkinThickness"][i] < 18.0:
            skin_level[i]="(7) P15th - P25th"

        elif pima["SkinThickness"][i] == 18.0:
            skin_level[i]="(8) P25th"

        elif pima["SkinThickness"][i] > 18.0 and pima["SkinThickness"][i] < 23.5:
            skin_level[i]="(9) P25th - P50th"

        elif pima["SkinThickness"][i] == 23.5:
            skin_level[i]="(10) P50th"

        elif pima["SkinThickness"][i] > 23.5 and pima["SkinThickness"][i] < 29.0:
            skin_level[i]="(11) P50th - P75th"

        elif pima["SkinThickness"][i] == 29.0:
            skin_level[i]="(12) P75th"

        elif pima["SkinThickness"][i] > 29.0 and pima["SkinThickness"][i] < 31.9:
            skin_level[i]="(13) P75th - P85th"

        elif pima["SkinThickness"][i] == 31.9:
            skin_level[i]="(14) P85th"

        elif pima["SkinThickness"][i] > 31.9 and pima["SkinThickness"][i] < 33.7:
            skin_level[i]="(15) P85th - P90th"

        elif pima["SkinThickness"][i] == 33.7:
            skin_level[i]="(16) P90th"

        elif pima["SkinThickness"][i] > 33.7 and pima["SkinThickness"][i] < 35.9:
            skin_level[i]="(17) P90th - P95th"

        elif pima["SkinThickness"][i] == 35.9:
            skin_level[i]="(18) P95th"

        elif pima["SkinThickness"][i] > 35.9:
            skin_level[i]="(19) >P95th"

    elif pima["Age"][i] >= 80.0:  #Only 1 woman is 81 years old
        if  pima["SkinThickness"][i] > 31.7:
            skin_level[i]="(20) >P95th"
  # Vì có 2 trường hợp thuộc nhóm >P95, nên thêm số thứ tự phía trước mỗi phân vị
  # để dễ nhìn và phân biệt từng trường hợp >P95 đó thuộc loại nào."

In [ ]:
# Thêm cột phân loại mức độ dày da
pima.insert(4, "Skin Level", skin_level)

In [ ]:
# Kiểm tra 5 dòng đầu
pima.head(5)

,Pregnancies,Glucose,Glucose Result,BloodPressure,Skin Level,SkinThickness,Insulin,BMI,Nutritional Status,DiabetesPedigreeFunction,Age,Outcome
0,6,148,Rối loạn dung nạp glucose,72,(17) P90th - P95th,35,0,33.6,Béo phì,0.627,50,1
1,1,85,Bình thường,66,(12) P75th,29,0,26.6,Thừa cân,0.351,31,0
2,8,183,Rối loạn dung nạp glucose,64,0 NA,0,0,23.3,Bình thường,0.672,32,1
3,1,89,Bình thường,66,(9) P25th - P50th,23,94,28.1,Thừa cân,0.167,21,0
4,0,137,Bình thường,40,(17) P90th - P95th,35,168,43.1,Béo phì,2.288,33,1


In [ ]:
# Thống kê số lượng phụ nữ thuộc từng nhóm phân vị độ dày da
pima['Skin Level'].value_counts()

,count
Skin Level,
0 NA,227
(19) >P95th,145
(11) P50th - P75th,87
(9) P25th - P50th,79
(15) P85th - P90th,50
(13) P75th - P85th,46
(17) P90th - P95th,23
(7) P15th - P25th,20
(8) P25th,20


**Nhận xét**:
+ Có 227 phụ nữ không có thông tin về độ dày da (SkinThickness = 0).
+ Phần lớn mẫu nghiên cứu có phân vị độ dày da > P95, cho thấy mức độ mỡ dưới da cao.
+ Có 53 phụ nữ có giá trị dưới P15, điều này có thể cho thấy họ đang thiếu dinh dưỡng.
+ Chỉ có 1 phụ nữ từ 80 tuổi trở lên, và người này có phân vị độ dày da > P95.


In [ ]:
# Đánh giá nguy cơ tiểu đường ở người có dấu hiệu suy dinh dưỡng qua độ dày da và BMI

# Số người mắc tiểu đường có độ dày da < 15.8 mm (nguy cơ suy dinh dưỡng theo SkinThickness)
so_nguoi_tieu_duong_da_mong = ((pima['SkinThickness'] < 15.8) & (pima['Outcome'] == 1)).sum()
print("Số người mắc tiểu đường có độ dày da < 15.8 mm:", so_nguoi_tieu_duong_da_mong)

# Số người mắc tiểu đường có BMI < 18.5 (nguy cơ suy dinh dưỡng theo BMI)
so_nguoi_tieu_duong_bmi_thap = ((pima['BMI'] < 18.5) & (pima['Outcome'] == 1)).sum()
print("Số người mắc tiểu đường có BMI < 18.5:", so_nguoi_tieu_duong_bmi_thap)

# Số người mắc tiểu đường có cả BMI < 18.5 và độ dày da < 15.8 mm (suy dinh dưỡng theo cả hai chỉ số)
so_nguoi_tieu_duong_bmi_skin_thap = ((pima['BMI'] < 18.5) & (pima['SkinThickness'] < 15.8) & (pima['Outcome'] == 1)).sum()
print("Số người mắc tiểu đường có cả BMI < 18.5 và độ dày da < 15.8 mm:", so_nguoi_tieu_duong_bmi_skin_thap)

Số người mắc tiểu đường có độ dày da < 15.8 mm: 94
Số người mắc tiểu đường có BMI < 18.5: 2
Số người mắc tiểu đường có cả BMI < 18.5 và độ dày da < 15.8 mm: 2


**Nhận xét**":
+ Có 94 người mắc tiểu đường có độ dày da thấp (<15.8 mm), cho thấy nguy cơ suy dinh dưỡng theo lớp mỡ dưới da.
+ Tuy nhiên, chỉ 2 người có BMI thấp (<18.5), cho thấy suy dinh dưỡng toàn thân là hiếm. Cả 2 người này đều có da mỏng và BMI thấp, cho thấy một nhóm nhỏ có suy dinh dưỡng rõ rệt.
+ Việc đo độ dày da khó chính xác và dễ bị ảnh hưởng bởi bệnh lý, nên cần thêm dữ liệu để đánh giá dinh dưỡng tốt hơn.